In [51]:
from __future__ import unicode_literals, print_function, division
from torch.utils.data import Dataset

import sys
sys.path.append("../../")
import time
import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf

import torch
from io import open
import glob
import string
import random
import torch.nn as nn
import math
import random
import urllib.request
import syft as sy 
from pathlib import Path

import unicodedata
import time
from syft.frameworks.torch.federated import utils

from syft.workers import WebsocketClientWorker

#Microsoft recommender
from recommenders.reco_utils.recommender.ncf.ncf_singlenode import NCF 
from recommenders.reco_utils.recommender.ncf.dataset import Dataset as NCFDataset 
from recommenders.reco_utils.dataset import movielens
from recommenders.reco_utils.dataset.python_splitters import python_chrono_split
from recommenders.reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                      recall_at_k, get_top_k_items)
from recommenders.reco_utils.common.constants import SEED as DEFAULT_SEED
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

EPOCHS = 100
BATCH_SIZE = 256

SEED = DEFAULT_SEED 

# loading movie leans data
df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE,header=["userID", "itemID", "rating", "timestamp"])

l_train = [12500, 25000, 37500, 50000, 62500, 75000, 87500, 100000]
l_mod = [0] + l_train + [max(l_train)+1]
list_of_dfs = [df.iloc[l_mod[n]:l_mod[n+1]] for n in range(len(l_mod)-1)]
print("movielens data partitioned")

100%|██████████| 4.81k/4.81k [00:03<00:00, 1.44kKB/s]


100000
       userID  itemID  rating  timestamp
0         196     242     3.0  881250949
1         186     302     3.0  891717742
2          22     377     1.0  878887116
3         244      51     2.0  880606923
4         166     346     1.0  886397596
...       ...     ...     ...        ...
12495     387     333     3.0  886479484
12496     175       9     4.0  877108146
12497     296     475     4.0  884196555
12498      62     180     4.0  879373984
12499     343       8     5.0  876404836

[12500 rows x 4 columns]


In [63]:

train = []
test = []
for i in range(len(list_of_dfs)-1):
    train_t, test_t = python_chrono_split(list_of_dfs[i], 0.75)
    train.append(train_t)
    test.append(test_t)
data = []
for i in range(len(train)):
    data.append(NCFDataset(train=train[i], test=test[i], seed=SEED))

model = []  # 8 NCF models containig the raspberry pi data
for i in range(len(train)):
    model.append(NCF(n_users=data[i].n_users,n_items=data[i].n_items,model_type="NeuMF",n_factors=4,layer_sizes=[16,8,4],\
            n_epochs=EPOCHS,batch_size=BATCH_SIZE,learning_rate=1e-3,verbose=10,seed=SEED))

print("NCF data models created")

[<recommenders.reco_utils.recommender.ncf.ncf_singlenode.NCF object at 0x7f86c2b84710>, <recommenders.reco_utils.recommender.ncf.ncf_singlenode.NCF object at 0x7f8738f05b38>, <recommenders.reco_utils.recommender.ncf.ncf_singlenode.NCF object at 0x7f86d542a390>, <recommenders.reco_utils.recommender.ncf.ncf_singlenode.NCF object at 0x7f86b048a8d0>, <recommenders.reco_utils.recommender.ncf.ncf_singlenode.NCF object at 0x7f86aaebee80>, <recommenders.reco_utils.recommender.ncf.ncf_singlenode.NCF object at 0x7f86a9763e48>, <recommenders.reco_utils.recommender.ncf.ncf_singlenode.NCF object at 0x7f86a9649e10>, <recommenders.reco_utils.recommender.ncf.ncf_singlenode.NCF object at 0x7f86d56797b8>]
movielens data partitioned


In [ ]:
#ip_address = ["10.1.1.141", "10.1.1.101"]  these are the ip address's of the raspberry pi's
id_names = ["piOne", "piTwo", "piThree", "piFour", "piFive", "piSix", "piSeven", "piEight"]
hook = sy.TorchHook(torch) 
#set up of remote workers

# piOne = WebsocketClientWorker(id=id_names[0], port=8777, **{"host": ip_address[0], "hook": hook, "verbose": args.verbose})
# piTwo = WebsocketClientWorker(id=id_names[1], port=8778, **{"host": ip_address[1], "hook": hook, "verbose": args.verbose})
# piThree = WebsocketClientWorker(id=id_names[2], port=8777, **{"host": ip_address[0], "hook": hook, "verbose": args.verbose})
# piFour = WebsocketClientWorker(id=id_names[3], port=8778, **{"host": ip_address[1], "hook": hook, "verbose": args.verbose})
# piFive = WebsocketClientWorker(id=id_names[4], port=8777, **{"host": ip_address[0], "hook": hook, "verbose": args.verbose})
# piSix = WebsocketClientWorker(id=id_names[5], port=8778, **{"host": ip_address[1], "hook": hook, "verbose": args.verbose})
# piSeven = WebsocketClientWorker(id=id_names[6], port=8777, **{"host": ip_address[0], "hook": hook, "verbose": args.verbose})
# piEight = WebsocketClientWorker(id=id_names[7], port=8778, **{"host": ip_address[1], "hook": hook, "verbose": args.verbose})

piOne = sy.VirtualWorker(hook, id=id_names[0])  
piTwo = sy.VirtualWorker(hook, id=id_names[1])  
piThree = sy.VirtualWorker(hook, id=id_names[2])  
piFour = sy.VirtualWorker(hook, id=id_names[3])  
piFive = sy.VirtualWorker(hook, id=id_names[4])  
piSix = sy.VirtualWorker(hook, id=id_names[5]) 
piSeven = sy.VirtualWorker(hook, id=id_names[6])  
piEight = sy.VirtualWorker(hook, id=id_names[7])

workers = [piOne, piTwo, piThree, piFour, piFive, piSix, piSeven, piEight]

print("virtual workers setup")

In [60]:
#pysyft arguments for federated learning
class Arguments():
    def __init__(self):
        self.batch_size = 1
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False
        self.context_size=3
        self.embedding_dim=10

In [65]:
args=Arguments()

In [66]:

# this is an attempt to create the federated train loader from the whole original dataset
training_data, testing_data = python_chrono_split(df, 0.75)
datasetAll = NCFDataset(train=training_data, test=testing_data, seed=SEED)

modelOfAllData = NCF(n_users=datasetAll.n_users,n_items=datasetAll.n_items,model_type="NeuMF",n_factors=4,layer_sizes=[16,8,4],\
            n_epochs=EPOCHS,batch_size=BATCH_SIZE,learning_rate=1e-3,verbose=10,seed=SEED)

federated_train_loader = sy.FederatedDataLoader(
            datasetAll
            .federate(workers),
            batch_size=args.batch_size)

#Unfortunately this code will not work as the dataset must be a federated dataset 

AttributeError: 'Dataset' object has no attribute 'federate'

In [ ]:
#This is meant to be the loop to perform the learning process
#but unfortunatelt the dataset is not federated and the NCF model hasnt been adjusted properly to work in a federated way

for e in range(args.epochs):
  for inputs, labels in federated_train_loader:
    # Location of current batch
    worker = inputs.location  # <---- Where will send the model to
    # Initialize hidden state
    h = torch.Tensor(np.zeros((BATCH_SIZE, HIDDEN_DIM)))
    # Send hidden state and model to current worker
    h = h.send(worker)   # <---- These steps are crucial
    modelOfAllData.send(worker)   # <---- for Federated Learning
    # Setting accumulated gradients to zero before backward step
#     optimizer.zero_grad()
    # Output from the model
#     output, _ = modelOfAllData(inputs, h)
#     # Calculate the loss and perform backprop
#     loss = criterion(output.squeeze(), labels.float())
#     loss.backward()
#     # Clipping the gradient to avoid explosion
#     nn.utils.clip_grad_norm_(model.parameters(), CLIP)
#     # Backpropagation step
#     optimizer.step() 
    
    # Get the model back to the local worker
    modelOfAllData.get() # <--- We need get the model back before sending to another worker

In [ ]:

#All cells below this are from the microsoft recommender 


# n_factors (int): Dimension of latent space.
# layer_sizes (list): Number of layers for MLP.

# training the model
model.fit(data)

predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]

# saving the predictions in a dataframe
predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()


In [52]:
users, items, preds = [], [], []
item = list(train.itemID.unique())
for user in train.userID.unique():
    user = [user] * len(item) 
    users.extend(user)
    items.extend(item)
    preds.extend(list(model.predict(user, item, is_list=True)))

all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)


hello
